# Определение стоимости автомобилей для сервиса «Не бит, не крашен»

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.


<a id='К-содержанию'></a>
**Содержание проекта** \
[Подготовка данных](#Подготовка-и-анализ-данных)\
[Проверка_корреляции данных](#Проверка-корреляции-данных)\
[Обучение моделей](#Обучение-моделей)\
[Общий вывод](#Общий-вывод)\
[Чек-лист_проверки](#Чек-лист-проверки)

In [ ]:
%%capture
!pip install phik
!pip install scipy=1.13.1
!pip install numpy=1.26.4
!pip install catboost
!pip install lightgbm
!pip install category_encoders
!pip install --upgrade scikit-learn seaborn

In [ ]:
import numpy as np
print(np.__version__)

import phik
print(phik.__version__)

import scipy
print(scipy.__version__)

import sklearn
print(sklearn.__version__)

In [ ]:
#импортируем библиотеки перед началом проекта одним блоком
import pandas as pd
import numpy as np
import random
import re
import math
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from scipy import stats as st
from IPython.display import display, Markdown

# загружаем класс pipeline
from sklearn.pipeline import Pipeline

# загружаем классы для подготовки данных
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from category_encoders import BinaryEncoder


# загружаем класс для работы с пропусками
from sklearn.impute import SimpleImputer

# импортируем классы GridSearchCV
from sklearn.model_selection import GridSearchCV

# загружаем нужные модели
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
import catboost as cb

# загружаем функцию для работы с метриками
from sklearn.metrics import mean_squared_error

# тесты для проверки корреляций
from phik import resources
from phik.report import plot_correlation_matrix
from phik import phik_matrix

import warnings
warnings.filterwarnings('ignore',category=UserWarning)

In [ ]:
# вынесем блок с настройками и функциями

# зададим стандарт датафрейма перед загрузкой
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', '{:.3f}'.format)
pd.options.mode.chained_assignment = None # warning при обращении к столбцам датафрейма утомил

try:
    pd.set_option('future.no_silent_downcasting', True)
except:
    pass

# выводим информацию по структуре и данным по каждому датафрейму
def get_info(dataset):
    display(dataset.info(), dataset.head(5), dataset.tail(5))


# вывод гистограммы и boxplot
def hist_box(data,column,hue=None):
    
    palette = sns.color_palette('hsv', n_colors=10)
    random_color = random.choice(palette)
    
    if hue == None:
        
        fig, axes = plt.subplots(1, 2, figsize=(7, 2)) 
        ax = axes[0]
        sns.histplot(data=data, x=column, bins=10, 
                     color=random_color,
                     alpha=0.6,
                     legend='auto', ax=ax)
    else:
        fig, axes = plt.subplots(1, 2, figsize=(7, 2)) 
        ax = axes[0]
        sns.histplot(data=data, x=column, bins=10, 
                     color=random_color,
                     alpha=0.6,
                     hue=hue, 
                     multiple='stack',
                     legend='auto', ax=ax)
        

    ax = axes[1]
    sns.boxplot(x=data[column], color=random_color, ax=ax)
    ax.set_ylabel('')

    plt.tight_layout()
    plt.show();


# функция группировки по километражу
def group(km):
    if km <= 135000:
        return ((km - 1) // 15000 + 1) * 15000
    else:
        return 150000

# функция подсчета абсолютных значений для указания на круговой диаграмме
def absolute_value(val,df):
    a = round(val/100.*df.sum())
    return a

# функция построения круговой диаграммы
def pieplot(data,column):
    data_grouped = data.groupby(column)[column].count()
    data.groupby(column)[column].count().plot(kind='pie', 
             title ='Признак {}\n'.format(column),
             radius=1.3,
             ylabel='',
             autopct=lambda x: f'{x:.1f}%',
             pctdistance=0.6 );

<a id='Подготовка-и-анализ-данных'></a> 
## Подготовка и анализ данных
[К содержанию](#К-содержанию) 

In [ ]:
# импортируем данные
try:
    autos = pd.read_csv('/Users/roman_yakovlev/Downloads/Практикум_DS/Проекты/Project_11_Autosales/autos.csv')
except:
    autos = pd.read_csv('/datasets/autos.csv')
    

In [ ]:
# посмотрим на данные
get_info(autos)

In [ ]:
# сразу приведем названия столбцов и тестовых записей к нижнему регистру
autos.columns = [i.lower() for i in autos.columns]

#приведем все текстовые значения к нижнему регистру
autos = autos.apply(lambda col: col.apply(lambda x: x.lower() if isinstance(x, str) else x))

autos.columns

In [ ]:
# приведем даты к типу datetime
columns_date = ['datecrawled','datecreated','lastseen']

for column in columns_date:
    autos[column]= pd.to_datetime(autos[column], format='%Y-%m-%d %H:%M:%S')

autos.info()

In [ ]:
# проверим уникальные значения в текстовых столбцах
columns_str = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand','repaired']


for column in columns_str:
    display(autos[column].unique())

In [ ]:
# для наглядности выведем количество пропусков в текстовых данных
display(autos[columns_str].isna().sum())

# заменим пропуски на категорию other, тем более эта категория в большинстве признаков и так уже есть
for column in columns_str:
    autos[column]=autos[column].fillna('other')

# и еще раз проверим
autos.info();

In [ ]:
# заменим опечатки и неявные дубликаты в значениях текстовых столбцов
autos['model'] = autos['model'].replace('rangerover', 'range_rover')
autos['fueltype'] = autos['fueltype'].replace({'gasoline': 'petrol', 'lpg': 'cng'})

In [ ]:
# в столбце с количеством фотографий похоже одни нули - проверим
print('Количество записей с отсутствующими фотографиями: ', autos[autos['numberofpictures'] == 0]['numberofpictures'].count())

In [ ]:
# удалим записи, в которых год регистрации указан после даты выгрузки анкеты, что явно некорректно
autos = autos[(autos['registrationyear'])<(autos['datecrawled'].dt.year)]

In [ ]:
# столбец с фотографиями неинформативен, лучше удалить, также как и столбцы с временными данными по загрузке анкеты и последней активности пользователя
# на цену эти технические параметры не влияют, в отличие от даты загрузки анкеты, т.к. цена во времени на один и тот же автомобиль может меняться

display(autos.head(3))
autos = autos.drop(columns=['datecrawled', 'numberofpictures', 'lastseen'])
autos.head(3)

In [ ]:
# проверим на дубликаты
print('Количество дубликатов: ',autos.duplicated().sum())

# уберем дубликаты
autos = autos.drop_duplicates()

In [ ]:
# посмотрим статистики и построим отдельно графики гистограммы и "ящик с усами" для количественных признаков

columns_num = ['price', 'power', 'kilometer']

display(autos[columns_num].describe())

print(f'Распределение количественных переменных в датасете autos: \n')    
for index, column in enumerate(columns_num):
    hist_box(autos,column);

По столбцу power видны аномальные значения в 0-10 лошадиных силы и более 500 и даже 1000 лошадиных сил, что крайне маловероятно для рыночных серийных автомобилей, посчитаем их количество.

In [ ]:
print('Количество автомобилей с аномальными значениями мощности двигателя: ', autos[(autos['power'] < 15) | (autos['power'] > 500)]['power'].count())

Таких записей достаточно много, игнорировать аномальные значения некорректно - это повлияет на качество обучения модели, удалить 10% записей также недопустимо, поэтому заменим значения мощности меньше 15 лошадиных сил или больше 500 в столбце ‘power’ на медианные значения, соответствующие модели и году регистрации прочих автомобилей в выборке.

In [ ]:
# зафиксируем условие для отбора некорректных строк
condition = ((autos['power'] < 15) | (autos['power'] > 500))

# отберем эти корректные строки для вычисления медиан по ним
valid_autos = autos[~condition]

# вычислим медианные значения для каждой группы (модель и год регистрации) без учета некорректных строк
medians = valid_autos.groupby(['model', 'registrationyear'])['power'].median()
medians = medians.astype(int)
medians_dict = medians.to_dict()

# заменим аномальные значени на медианные
autos.loc[condition, 'power'] = autos.loc[condition].apply(
    lambda row: medians_dict.get((row['model'], row['registrationyear']), row['power']), axis=1)


In [ ]:
print('Количество автомодилей с аномальными значениями мощности двигателя: ', autos[(autos['power'] < 15) | (autos['power'] > 500)]['power'].count())

palette = sns.color_palette('hsv', n_colors=10)
random_color = random.choice(palette)
plt.figure(figsize=(7, 2))
sns.boxplot(x=autos['power'], color=random_color)
plt.ylabel('')
plt.title('Распределение значений по столбцу power')
plt.tight_layout()
plt.show()

После вышеуказанной операции у нас остался небольшой пул записей с некорректными данными, для которых не нашлось группы (и соответственно медианного значения) по параметрам года регистрации и модели, для них проведем аналогичную операцию но посчитаем медиану только по модели, что менее точно, но более релевантно относительно 0 или 5 000 лошадиных сил.

In [ ]:
# вычисление медианных значений для каждой группы (модель) без учета некорректных строк
medians = valid_autos.groupby('model')['power'].median()
medians = medians.astype(int)
medians_dict = medians.to_dict()

autos.loc[condition, 'power'] = autos.loc[condition].apply(
    lambda row: medians_dict.get((row['model']), row['power']), axis=1)

In [ ]:
autos[(autos['power'] < 15) | (autos['power'] > 500)]

Осталось две модели, для которых просто нет иных вариантов, их заполним вручную медианными значениями по записям, сгруппированным по бренду и году.

In [ ]:
autos.loc[280216,'power'] = autos[(autos['brand'] == 'land_rover')&(autos['registrationyear'] == 1970)&(autos['power'] != 0)]['power'].median()
autos.loc[234296,'power'] = autos[(autos['brand'] == 'land_rover')&(autos['registrationyear'] == 1978)&(autos['power'] != 0)]['power'].median()

По ценам также наблюдаются аномалии с минималными ценами на автомибили на уровне 0-100, посчитаем их.

In [ ]:
print('Количество автомодилей с аномальными значениями цен: ', autos[(autos['price'] < 100)]['price'].count())

palette = sns.color_palette('hsv', n_colors=10)
random_color = random.choice(palette)
plt.figure(figsize=(7, 2))
sns.boxplot(x=autos['price'], color=random_color)
plt.ylabel('')
plt.title('Распределение значений по столбцу price')
plt.tight_layout()
plt.show()

Заменим аномально низкие цены на медианные по аналогичным категориям автомобилей, единственно для точности добавим категории по километражу, т.к. пробег важен при оценке и учтем эти группы для расчета медианной цены. 

In [ ]:
autos['kilometer_group'] = autos['kilometer'].apply(group)

In [ ]:
# зафиксируем условие для отбора некорректных строк
condition = (autos['price'] < 100) 

# отберем эти корректные строки для вычисления медиан по ним
valid_autos = autos[~condition]

# вычислим медианные значения для каждой группы (модель, год регистрации, километраж) без учета некорректных строк
medians = valid_autos.groupby(['model', 'registrationyear','kilometer_group'])['price'].median()
medians = medians.astype(int)
medians_dict = medians.to_dict()

# заменим аномальные значени на медианные
autos.loc[condition, 'price'] = autos.loc[condition].apply(
    lambda row: medians_dict.get((row['model'], row['registrationyear'],row['kilometer_group']), row['price']), axis=1)

In [ ]:
autos[(autos['price'] < 100)]['price'].count()

In [ ]:
# по автомобилям для которых не нашлось групп вычислим медианные значения для групп по модели и году регистрации
medians = valid_autos.groupby(['model', 'registrationyear'])['price'].median()
medians = medians.astype(int)
medians_dict = medians.to_dict()

# заменим аномальные значени на медианные
autos.loc[condition, 'price'] = autos.loc[condition].apply(
    lambda row: medians_dict.get((row['model'], row['registrationyear']), row['price']), axis=1)

In [ ]:
autos[(autos['price'] < 100)]['price'].count()

In [ ]:
# по автомобилям для которых не нашлось и таких групп вычислим медианные значения просто по модели
medians = valid_autos.groupby(['model'])['price'].median()
medians = medians.astype(int)
medians_dict = medians.to_dict()

# заменим аномальные значени на медианные
autos.loc[condition, 'price'] = autos.loc[condition].apply(
    lambda row: medians_dict.get((row['model']), row['price']), axis=1)

In [ ]:
autos[(autos['price'] < 100)]['price'].count()

In [ ]:
# посмотрим на данные с учетом изменений
for index, column in enumerate(columns_num):
    hist_box(autos,column);

Посмотрим на категориальные признаки, по 4-м из них: vehicletype, gearbox, fueltype, repaired, построим круговые диаграммы, по остальным registrationyear, model, brand, datecreated, postalcode, kilometer_group - слишком много значений для круговой диаграммы, рассмотрим их с помощью столбчатых графиков и гистограмм.

In [ ]:
#построим графики типа pie по категориальным данным из первой части пункта выше
columns_cat_1 = ['vehicletype', 'gearbox','fueltype','repaired']

plt.figure(figsize=[10, 6], dpi=120)
plt.subplots_adjust(wspace=0.1, hspace=0.3)
plt.suptitle('Соотношение классов по категориальным признакам', y=1)

for i,column in enumerate(columns_cat_1):
    
    plt.subplot(2,2,i+1)
    pieplot(autos,column);

In [ ]:
# построим гистограму по годам выпуска автомобилей

plt.figure(figsize=[5, 3])
autos['registrationyear'].hist(bins=20);

In [ ]:
# выявили некорреткные значения годов выпуска, посмотрим на все уникальные значения
display(np.sort(autos['registrationyear'].unique()))

# посчитаем количество записей с некорректнымми данными - автомобилями с датой выпуска менее 1915 и страше 2019 годов
print('\nКоличество записей с некорректными годами выпуска: ', autos[(autos['registrationyear']<1920)|(autos['registrationyear']>2019)]['registrationyear'].count())

# заменим некорреткные годы на 0
autos.loc[(autos['registrationyear']<1920)|(autos['registrationyear']>2019),'registrationyear'] = 0

# посмотрим распределение с учетом корреткировки
plt.figure(figsize=[15, 3])
autos['registrationyear'].value_counts().sort_index().plot(kind='bar')
plt.xlabel('registrationyear')
plt.ylabel('count')
plt.title('Количество зарегистрированных машин в год')
plt.show()

In [ ]:
# посчитаем количество уникальных значений моделей
print('Количество уникальных значений моделей: ', autos['model'].nunique())

# посмотрим на распределение авто по моделям для первых ТОП100 
plt.figure(figsize=[16, 3])
autos['model'].value_counts().head(100).plot(kind='bar')
plt.xlabel('model')
plt.ylabel('count')
plt.title('Количество автомобилей для ТОП100 моделей')
plt.show()

In [ ]:
# посмотрим на распределение автомобилей по месяцам
plt.figure(figsize=[10, 3])
autos['registrationmonth'].value_counts().sort_index().plot(kind='bar')
plt.ylabel('count')
plt.title('Количество зарегистрированных автомобилей по месяцам')
plt.show();

In [ ]:
# посчитаем количество уникальных значений брендов
print('Количество уникальных значений моделей: ', autos['brand'].nunique())

# посмотрим на распределение авто по брендов
plt.figure(figsize=[12, 3])
autos['brand'].value_counts().plot(kind='bar')
plt.xlabel('model')
plt.ylabel('count')
plt.title('Количество автомобилей в разбивке по брендам')
plt.show()

In [ ]:
# посчитаем уникальные значения дат объявлений
print('Количество дат объявлений о продаже: ', autos['datecreated'].nunique())

# посмотрми распределение по датам обявлений
plt.figure(figsize=[17, 2])
plt.title('Количество объявлений по датам публикации')
autos['datecreated'].dt.to_period('d').value_counts().sort_index().plot(kind='bar');

# посмотрми распределение дат с группировкой по месяцам
plt.figure(figsize=[10, 2])
autos['datecreated'].dt.to_period('M').value_counts(ascending=True).sort_index().plot(kind='bar')
plt.show();

In [ ]:
# посчитаем уникальные значения почтовых кодов объявлений
print('Количество почтовых кодов в объявлениях: ', autos['postalcode'].nunique())

# посмотрим на распределение почтовых кодов для первых ТОП100 
plt.figure(figsize=[16, 3])
autos['postalcode'].value_counts().head(100).plot(kind='bar')
plt.xlabel('postalcode')
plt.ylabel('count')
plt.title('Количество почтовых кодов для ТОП100 моделей')
plt.show()

In [ ]:
# посмотрим на распределение автомобилей по группам километража
plt.figure(figsize=[9, 3])
autos['kilometer_group'].value_counts().sort_index().plot(kind='bar')
plt.xlabel('kilometer_group')
plt.ylabel('count')
plt.title('Количество автомобилей по группам километража пробега')
plt.show()

**Вывод**  
После импорта данных сразу привели названия столбцов и текстовых значений к нижнему регистру по всем столбцам, а также значения дат - к типу datetime.  
Далее:
- проанализировали значения текстовых признаков 'vehicletype', 'gearbox', 'model', 'fueltype', 'brand','repaired' - пропуски в них заменили на категорию 'other', чтобы не потерять эти записи при анализе, и учитывая наличие категории 'other' в большинстве указанных признаков;
- скорректировали опечатки и неявные дубликаты в значениях текстовых столбцов 'model', 'fueltype';
- удалили записи, в которых год регистрации указан после даты выгрузки анкеты, что явно некорректно;
- столбец с фотографиями неинформативен, поскольку заполнен только нулями - удалили из датасета, также как и столбцы с временными данными по загрузке анкеты и последней активности пользователя - на цену эти технические параметры не влияют;
- удалили порядка 9 тысяч явных дубликатов;
- построили графики гистограмм и "ящик с усами" для количественных признаков, что позволило выявить выбросы в столбце с мощностью двигателя (менее 15 и более 500 лошадиных сил) общим количеством 40287;
- заменили вышеуказанные аномалии несколькими итерациями, сгруппировав данные по модели и году выпуска, а потом просто по модели или бренду;
- по ценам также выявили аномалии с минимальными ценами на автомибили на уровне 0-100 в количестве 13035, которые, аналогично мощности ранее, скорректировали на медианные значения, сгруппировав данные по моделям, годам выпуска и километражу (добавили отдельный признак с категориями по километражу на каждые 15 тыс. километров пробега);
- по категориальным признакам 'vehicletype', 'gearbox', 'fueltype', 'repaired' построили круговые диаграммы, из которых обратили внимание на признак типа топлива - подавляющее большинство моделей 88,8% на бензине, следующая категория 'other', в которую были включены и неизвестные данные по этому признаку. Следующие за этими двумя категориями - газ, гибриды, электробатареи суммарно формируют менее 2% автомобилей, в таком случае полезность признака для обучения модели сомнительна, т.к. один из классов присутствует в подавляющем большинстве записей, а второй включает 'прочие', что не позволит эффективно оценивать вклад каждого класса при обучении модели. При обучении модели имеет смысл протестировать исключение данного признака из обучающей выборки.
- при анализе данных по годам выпуска выявили 266 записей с аномальными значениями (менее 1920 года и более 2019), заменили их на 0, в результате основная масса автомобилей по году выпуска находится в промежутке с 1980 по 2018 годы, остальные годы соответтсвуют редким и раритетным экземплярам;
- при анализе по месяцам регистрации автомобилей выявлена значительная часть автомобилей с неуказанным месяцем регистрации (0 на графике), остальные автомобилди распределены равномерно по всем месяцам;
- частотный анализ по моделям и брендам показал только самую крупную группу моделей 'other', включающую отсутствующие данные, остальные данные по указаным признакам распределены убывающим порядком от наиболее известных и популярных в продажах брендов(VW, Opel, BMW, MB, Audi, Ford) и их моделей (golf, 3er, corsa, astra, passat, a4, c-klasse, e-klasse);
- анализ дат размещения объявлений показал, что подавляющая масса объявлений была опубликована в течение марта и апреля 2016 года, то есть данные выгружены за месяц с начала марта по начало апреля;
- количество локаций по почтовым кодам объявлений составляет 8143 и распределение равномерно убывающее, в соответствии с графиком первых 100 кодов;
- согласно синтетическому признаку с категориями по километражу - больше половины автомобилей имеют пробег свыше 135 тысяч километров, это самая крупная категория. При обучении моделей можно будет протестировать и оценить вклад данного признака и, возможно, исключить его.


<a id='Проверка-корреляции-данных'></a> 
## Проверка корреляции данных
[К содержанию](#К-содержанию) 

In [ ]:
# построим коррелляционые матрицы
columns_num = ['price', 'power', 'kilometer']
# так как у нас данные распределены ненормально используем коэффициент Спирмена
plt.figure(figsize=(10,8), dpi= 80)
sns.heatmap(autos[columns_num].corr(method='spearman'), 
                        xticklabels=autos[columns_num].corr(method='spearman').columns, 
                        yticklabels=autos[columns_num].corr(method='spearman').columns, 
                        cmap='coolwarm' , 
                        center=0, 
                        annot=True)

plt.title('Коэффициенты корреляции Спирмена между количественными признаками\n', fontsize=16)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12, rotation='horizontal')
plt.show()


In [ ]:
# вычисление корреляционной матрицы с использованием коэффициента Phi_K
phik_matrix = autos.phik_matrix(interval_cols=['price', 'power', 'kilometer'])

# сразу будем смотреть heatmap
plt.figure(figsize=(13,11), dpi= 80)
sns.heatmap(phik_matrix.values, xticklabels=phik_matrix.columns, yticklabels=phik_matrix.index, cmap='coolwarm', center=0, annot=True)

plt.title('Коэффициенты корреляции Phi_K между всеми признаками\n', fontsize=18)
plt.xticks(fontsize=12, rotation=45, ha='right')
plt.yticks(fontsize=12, rotation='horizontal')
plt.show()

In [ ]:
autos = autos.drop(columns=['datecreated', 'fueltype', 'kilometer_group','registrationmonth'])

autos.head(2)

**Вывод**

Корреляция целевого признака с количественными признаками есть на уровне коэффициента Спирмена -0.33 с километражом и 0.51 с мощностью двигателя.
Проверка корреляции между всеми признаками по методу Phi_K показала связи с целевым признаком большинства входных признаков, кроме даты создания объявления: 
1) т.к. у нас выгрузка объявлений была по сути за один месяц - разброс дат был незначительным, для таких объявлений как продажа машин, цены на авто в таких промежутках обычно достаточно стабильны, если бы объявления были за год, то уже возможно было бы наблюдать изменения в связи с изменениями в экономике и на рынке в целом;  
2) признак типа топлива, как ранее отмечалось, малоинформативен, корреляция у него минимальна - исключаем;  
3) также и месяц регистрации авто очень слабокоррелирован с ценой, что логично - при покупке авто обращают внимание на год, а не месяц, тем более в этом признаке присутствует значительный шумный класс, поэтому признак также исключаем; 
Ранее сформированный синтетический признак километража по группам показал ту же связь, что и основной признак километража, более того они абсолютно коррелированы, поэтому от него можно избавиться для повышения качества обучения моделей.  
Также выявили крайне высокую корреляцию между входными признаками бренда, модели и типа авто, что логично, т.к. конкретные типы авто и моделей соответствуют конкретным брендам, линейки авто у брендов не пересекаются. Учитывая значительный пул значений other по признаку моделей и типу авто, это будет усложнять обучение модели, и тут наличие бренда скорее сыграет положительную роль - поэтому исключать какие-либо из этих признаков не будем.


In [ ]:
# подсчет частоты значений в столбце модели
value_counts = autos['model'].value_counts()

# фильтрация значений, которые встречаются реже 10 раз - иначе при обучении вылетают ошибки отсутствия категорий
filtered_values = value_counts[value_counts >= 10].index

# фильтрация DataFrame на основе этих значений
autos = autos[autos['model'].isin(filtered_values)]

autos.shape

<a id='Обучение-моделей'></a> 
## Обучение моделей
[К содержанию](#К-содержанию) 

Сначала сформируем наборы данных для обучения и теста моделей, далее сформируем пайплайны для подготовки данных и обучения моделей.

In [ ]:
# задаем константы
RANDOM_STATE = 42
TEST_SIZE = 0.15


In [ ]:
# сформируем наборы данных для обучения и тестов
X_train, X_test, y_train, y_test = train_test_split(
    autos.drop('price', axis=1),
    autos['price'],
    test_size = TEST_SIZE,
    stratify=autos['model'],
    random_state = RANDOM_STATE)

display(X_train.shape,y_train.shape)
display(X_train.head(),y_train.head())

In [ ]:
# создаём списки с названиями признаков
be_columns = ['vehicletype', 'gearbox', 'postalcode', 'model', 'brand', 'repaired']
num_columns = ['registrationyear', 'power', 'kilometer']

# создаём пайплайн для подготовки признаков из списка ohe_columns: заполнение пропусков и OHE-кодирование
be_pipe = Pipeline(
    [('simpleImputer_be', SimpleImputer(missing_values=np.nan, strategy='most_frequent')),
     ('be', BinaryEncoder())
    ]
    )

# создаём общий пайплайн для подготовки данных
data_preprocessor= ColumnTransformer(
    [('be', be_pipe, be_columns),
     ('num', StandardScaler(), num_columns)
    ], 
    remainder='passthrough'
)

In [ ]:
# посчитаем количество признаков после предварительной обработки данных
max_features = data_preprocessor.fit_transform(X_train).shape[1]
display(max_features)

# посмотрим названия столбцов после кодирования
display(data_preprocessor.get_feature_names_out())


In [ ]:
%%time

# создаём итоговый пайплайн для простой Линейной регрессии: подготовка данных и модель
pipe_final = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', LinearRegression())
])

# задаем "пустой" набор гиперпараметров для обучения при стандартных настройках модели
param_grid = [

    # словарь для модели LinearRegression()
    {
        'models': [LinearRegression()],
        'preprocessor__num': [StandardScaler()]
    }
]


# перебор с помощью GridSearchCV
grid_search_0 = GridSearchCV(
    pipe_final, 
    param_grid, 
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# обучим модель
model_0 = grid_search_0.fit(X_train, y_train)

# выгрузим метрику negMSE и преобразуем в RMSE
best_score_neg_mse_0 = model_0.best_score_
best_score_rmse_0 = round(float(np.sqrt(-best_score_neg_mse_0)),3)

In [ ]:
%%time

# создаём итоговый пайплайн для дерева решений: подготовка данных и модель
pipe_final_1 = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', DecisionTreeRegressor())
])

# задаем "пустой" набор гиперпараметров для обучения при стандартных настройках модели
param_grid_1 = [

     # словарь для модели DecisionTreeRegressor()
    {
        'models': [DecisionTreeRegressor(random_state=RANDOM_STATE)],
        'preprocessor__num': [StandardScaler()]
    }
]


# перебор с помощью GridSearchCV
grid_search_1 = GridSearchCV(
    pipe_final_1, 
    param_grid_1, 
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# обучим модель
model_1 = grid_search_1.fit(X_train, y_train)

# выгрузим метрику negMSE и преобразуем в RMSE
best_score_neg_mse_1 = model_1.best_score_
best_score_rmse_1 = round(float(np.sqrt(-best_score_neg_mse_1)),3)

In [ ]:
%%time

# создаём итоговый пайплайн для LGBM: подготовка данных и модель
pipe_final_2 = Pipeline([
    ('preprocessor', data_preprocessor),
    ('models', LGBMRegressor(random_state=RANDOM_STATE,verbosity=-1))
])

# задаем "пустой" набор гиперпараметров для обучения при стандартных настройках модели
param_grid_2 = [

     # словарь для модели LGBMRegressor()
    {
        'preprocessor__num': [StandardScaler()]
    }
]

# перебор с помощью GridSearchCV
grid_search_2 = GridSearchCV(
    pipe_final_2, 
    param_grid_2, 
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# обучим модель
model_2 = grid_search_2.fit(X_train, y_train)

# выгрузим метрику negMSE и преобразуем в RMSE
best_score_neg_mse_2 = model_2.best_score_
best_score_rmse_2 = round(float(np.sqrt(-best_score_neg_mse_2)),3)

In [ ]:
%%time

# итоговый пайплайн для CatBoost не формируем: 
# подготовка данных не требуется - подаем исходные наборы X_train, y_train и модель одна

estimator = CatBoostRegressor(silent=True)
cat_features = ['vehicletype', 'gearbox', 'model', 'brand', 'repaired','postalcode']

# задаем пока "пустой" набор гиперпараметров
param_grid_3 = [

     # словарь для модели CatBoostRegressor()
    {
        'iterations': [100] #по дефолту 1000 деревьев, скорректировал на 100 как у LGBM
    }
]

# перебор гиперпараметров с помощью GridSearchCV
grid_search_3 = GridSearchCV(
    estimator, 
    param_grid_3, 
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

# обучим модель
model_3 = grid_search_3.fit(X_train, y_train,cat_features=cat_features)

# выгрузим метрику negMSE и преобразуем в RMSE
best_score_neg_mse_3 = model_3.best_score_
best_score_rmse_3 = round(float(np.sqrt(-best_score_neg_mse_3)),3)

<a id='Анализ-моделей'></a> 
## Анализ моделей
[К содержанию](#К-содержанию) 

Простые модели LinearRegression и DecisionTreeRegression при стандартных параметрах показали более слабые результаты по метрике RMSE, 3258 и 3093 соответственно, линейная регрессия не выполнила требований относительно целевого ориентира в 2500 евро, а решающее дерево превзошло его незначительно:


In [ ]:
print(f'Метрика RMSE модели линейной регрессии с учетом кросс-валидации: {best_score_rmse_0}')
print(f'Время обучения модели линейной регрессии: {grid_search_0.cv_results_["mean_fit_time"][0]} с.')
print(f'Время предсказания модели линейной регрессии: {grid_search_0.cv_results_["mean_score_time"][0]} с.')

In [ ]:
print(f'Метрика RMSE модели решающего дерева с учетом кросс-валидации: {best_score_rmse_1}')
print(f'Время обучения модели решающего дерева: {grid_search_1.cv_results_["mean_fit_time"][0]} с.')
print(f'Время предсказания модели решающего дерева: {grid_search_1.cv_results_["mean_score_time"][0]} с.')

Ансамбли решающих деревьев LightGBM и CatBoost показали значительно более сильные результаты 1762 и 1707 соответственно:


In [ ]:
print(f'Метрика RMSE модели LightGBM с учетом кросс-валидации: {best_score_rmse_2}')
print(f'Время обучения модели LightGBM: {grid_search_2.cv_results_["mean_fit_time"][0]} с.')
print(f'Время предсказания модели LightGBM: {grid_search_2.cv_results_["mean_score_time"][0]} с.')

In [ ]:
print(f'Метрика RMSE модели CatBoost с учетом кросс-валидации: {best_score_rmse_3}')
print(f'Время обучения модели CatBoost: {grid_search_3.cv_results_["mean_fit_time"][0]} с.')
print(f'Время предсказания модели CastBoost: {grid_search_3.cv_results_["mean_score_time"][0]} с.')

По целевой метрике RMSE лучший результат у ансабля CatBoost (1707 евро), при этом по времени обучения Catboost (8.8 секунд) уступил более простым моделям линейной регрессии (6 секунд) и решающего дерева(7.6 секунд), а по времени прогнозирования также обошел все прочие модели с результатом 0.1 секунда. 

Учитываая значительное преимущество по целевой метрике качества и скорости прогнозирования, для дальнейшего тестирования оставим именно CatBoost. Для указанной модели проверим несколько гиперпараметров и лучший вариант проверим на тестовом наборе данных.

In [ ]:
%%time

# итоговый пайплайн для CatBoost не формируем: 
# подготовка данных не требуется - подаем исходные наборы X_train, y_train и модель одна

estimator = CatBoostRegressor(silent=True)
cat_features = ['vehicletype', 'gearbox', 'model', 'brand', 'repaired','postalcode']

# задаем пока "пустой" набор гиперпараметров
param_grid_4 = [

     # словарь для модели CatBoostRegressor()
    {
        'depth': [6,10],
        'iterations': [50, 100]
    }
]

# перебор гиперпараметров с помощью GridSearchCV
grid_search_4 = GridSearchCV(
    estimator, 
    param_grid_4, 
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=False,
    n_jobs=-1
)

# обучим модель
model_4 = grid_search_4.fit(X_train, y_train,cat_features=cat_features)

# выгрузим метрику negMSE и преобразуем в RMSE
best_score_neg_mse_4 = model_4.best_score_
best_score_rmse_4 = round(float(np.sqrt(-best_score_neg_mse_4)),3)

In [ ]:
print(f'Метрика RMSE модели CatBoost с учетом кросс-валидации: {best_score_rmse_4}')
print(f'Время обучения модели CatBoost: {grid_search_4.cv_results_["mean_fit_time"].mean()} с.')
print(f'Время предсказания модели CastBoost: {grid_search_4.cv_results_["mean_score_time"].mean()} с.')
print(f'Лучшие параметры модели CastBoost: {grid_search_4.best_params_}')

In [ ]:
# проверим работу модели на тестовой выборке
y_test_pred4 = model_4.predict(X_test)
print(f'Метрика RMSE на тестовой выборке: {round(np.sqrt(mean_squared_error(y_test, y_test_pred4)),3)}')

In [ ]:
# посмотрим на остатки по модели
fig, axes = plt.subplots(2,1,figsize=(6, 6))
fig.suptitle('Распределение остатков по прогнозам модели CatBoost на тестовой выборке', y=1)

ost = y_test - y_test_pred4
ax = axes[0]
sns.histplot(data=ost, ax=ax)

ax = axes[1]
sns.scatterplot(x=y_test_pred4, y=y_test - y_test_pred4,ax=ax);

plt.tight_layout();
plt.show();

In [ ]:
# посмотрим важность признаков
feature_importances = grid_search_4.best_estimator_.get_feature_importance()
feature_names = X_train.columns

# соберем в dataFrame и отсортируем по убыванию
feature_importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importances
})

feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

# изуализируйте важность признаков с помощью seaborn
plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance_df)
plt.xlabel("Важность признаков")
plt.ylabel("Признаки")
plt.title("Важность признаков в модели CatBoost на тестовой выборке")
plt.show()

C лучшими параметрами ('depth': 10, 'iterations': 100) модель CatBoost на тестовой выборке показала результат целевой метрики RMSE 1660 евро, еще улучшив предсказательную способность относительно стандартного набора параметров.

Анализ остатков показал, что они распределены скорее приближенно к распределению Стьюдента, без нескольких пиков и выбросов, при этом диаграмма рассеяния остатков показывают наличие явных зависимостей от прогнозных значений, то есть не все закономерности были выявлены на стадии обучения и остатки распределены неслучайным образом, скорее всего это результатат того, что данные были сформированы синтетическим образом. 
Анализ важности признаков подчеркнул вклад года выпуска авто, мощности двигателя, бренда, типа кузова и модели авто; пробег, тип трансмиссии и место продажи - менее значимы.

<a id='Общий-вывод'></a> 
## Общий вывод
[К содержанию](#К-содержанию) 

После импорта данных была проведена **предобработка данных** и заполнены пропуски в признаках 'vehicletype', 'gearbox', 'model', 'fueltype', 'brand','repaired' категориями 'other'.
Далее скорректировали опечатки и неявные дубликаты, удалили столбец с фотографиями, поскольку он заполнен только нулями, и столбцы с временными данными по загрузке анкеты и последней активности пользователя, т.к. на цену эти технические параметры не влияют. Также были исключены записи с годом регистрации авто после года подачи заявления - явно ошибочные, при этом год регистрации авто - один из основных параметров прогнозирования цены (в дальнейшем анализ обученных моделей это подтвердил),  после чего удалили порядка 9 тысяч явных дубликатов.  
**Исследовательский анализ** с применением графиков гистограмм и "ящик с усами" для количественных признаков позволил выявить выбросы в столбце с \_мощностью\_ двигателя (менее 15 и более 500 лошадиных сил) общим количеством 40287. Эти аномалии заменили медианными значениями, сгруппировав данные по модели и году выпуска, а на следующих итерациях только по модели или бренду.  
По \_ценам\_ также выявили аномалии с минимальными ценами на автомибили на уровне 0-100 в количестве 13035, которые аналогично мощности скорректировали на медианные значениями, сгруппировав данные по моделям, годам выпуска и километражу (добавили отдельный признак с категориями по километражу на каждые 15 тыс. километров пробега).  
По \_категориальным признакам\_ 'vehicletype', 'gearbox', 'fueltype', 'repaired' построили круговые диаграммы, из которых обратии внимание на признак типа топлива - подавляющее большинство моделей 88,8% на бензине, следующая категория 'other', в которкую были включены и неизвестные данные по эту признаку, следующие за этими двумя категориями - газ, гибриды, электробатареи суммарно формируют менее 2% автомобилей, в таком случае полезность признака для обучения модели сомнительна, что в дальнейшем подтвердила и очень низкая корреляция с целевым признаком - признак типа топлива был удален.   
При анализе данных по \_годам выпуска\_ выявили 266 записей с аномальными значениями (менее 1920 года и более 2019), заменили их на 0, в результате выяснили, что основная масса автомобилей по году выпуска находится в промежутке с 1980 по 2018 годы, остальные годы соответствуют редким и раритетным экземплярам.  
При анализе распределения по \_месяцам регистрации\_ автомобилей выявлена значительная часть автомобилей с неуказанным месяцем регистрации (0 на графике), остальные автомобили распределены равномерно по всем месяцам, и учитывая, что месяц первичной регистрации авто мало влияет на цену (в отличие от года), а также выявленнную в дальнейшем низкую корреляцию признака с целевым - данный признак также было принято исключить из обучающего датасета.  
Частотный анализ по \_моделям\_ и \_брендам\_ показал только самую крупную группу моделей 'other', включающую отсутствующие данные, остальные данные по указаным признакам распределены убывающим порядком от наиболее известных и популярных в продаже брендов(VW, Opel, BMW, MB, Audi, Ford) и их моделей (golf, 3er, corsa, astra, passat, a4, c-klasse, e-klasse).  
Анализ \_дат размещения объявлений\_ показал, что подавляющая масса объявлений была опубликована в течение марта и апреля 2016 года, то есть данные выгружены за месяц с начала марта по начало апреля - в таком случае влияние даты размещения объявления на цену минимально, т.к. рынок более инерционен и дата объявления может быть информативна на более длительных промежутках (от квартала и более), корреляции данного признака с целевым околонулевая - поэтому данный признак далее также исключили.  
Количество локаций по \_почтовым кодам\_ объявлений составляет 8143 и распределение равномерно убывающее, корреляция данного признака также была выявлена как околонулевая - признак был исключен.

**Корреляция** целевого признака с количественными признаками есть на уровне коэффициента Спирмена -0.33 с километражом пробега и 0.51 с мощностью двигателя. Проверка корреляции между всеми признаками по методу Phi_K показала связи с целевым признаком большинства входных признаков, кроме даты создания объявления и очень низким уровнем корреляции с признаками типа топлива, месяца регистрации авто. Ранее сформированный синтетический признак километража по группам показал ту же связь, что и основной признак километража, более того они абсолютно коррелированы, поэтому от него также избавились для повышения качества обучения моделей.
Также выявили крайне высокую корреляцию между входными признаками бренда, модели и типа авто, что логично, т.к. конкретные типы авто и моделей соответствуют конкретным брендам, линейки авто у брендов не пересекаются. Учитывая значительный пул значений 'other' по признаку моделей и типу авто, это будет усложнять обучение модели, и тут наличие бренда скорее сыграет положительную роль - поэтому исключать какие-либо из этих признаков не стали.

При **обучении моделей** подготовили простые модели линейной регресии(со стандартными параметрами), и решающего дерева (также со стандартными параметрами), а также ансамбли решающих деревьев LightGBM и CatBoost. 
По целевой метрике RMSE лучший результат у ансабля CatBoost (1707 евро), при этом по времени обучения Catboost (8.8 секунд) уступил более простым моделям линейной регрессии (6 секунд) и решающего дерева(7.6 секунд), а по времени прогнозирования также обошел все прочие модели с результатом 0.1 секунда.

Учитывая значительное преимущество по целевой метрике качества и скорости прогнозирования, для дальнейших настройки и тестирования оставили именно CatBoost. 

**C лучшими параметрами ('depth': 10, 'iterations': 100) модель CatBoost на тестовой выборке показала результат целевой метрики RMSE 1660 евро, еще улучшив предсказательную способность относительно стандартного набора параметров.**

Анализ остатков показал, что они распределены скорее приближенно к распределению Стьюдента, без нескольких пиков и выбросов, при этом диаграмма рассеяния остатков показывают наличие явных зависимостей от прогнозных значений, то есть не все закономерности были выявлены на стадии обучения и остатки распределены неслучайным образом, скорее всего это результатат того, что данные были сформированы синтетическим образом. Анализ **важности признаков** подчеркнул вклад года выпуска авто, мощности двигателя, бренда, типа кузова и модели авто; пробег, тип трансмиссии и место продажи - менее значимы.

<a id='Чек-лист-проверки'></a> 
## Чек-лист проверки
[К содержанию](#К-содержанию) 

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнена загрузка и подготовка данных
- [ ]  Выполнено обучение моделей
- [ ]  Есть анализ скорости работы и качества моделей